In [55]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [56]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [57]:
# Install these packages with these specific versions else the notebook breaks
!pip install transformers==4.5.1
!pip install pytorch_lightning==1.2.10
!pip install sentencepiece

In [58]:
# Import packages
import argparse
import glob
import pickle
import os
import json
import time
import logging
import random
import re
from tqdm import tqdm
from itertools import chain
from string import punctuation
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize


import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import sentencepiece


from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
import csv
from dataclasses import dataclass

from enum import Enum
from typing import List, Optional
from transformers import PreTrainedTokenizer

In [60]:
def set_seed(seed):
  random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

In [61]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
  def on_validation_end(self, trainer, pl_module):
    logger.info("***** Validation results *****")
    if pl_module.is_logger():
      metrics = trainer.callback_metrics
      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "val_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

  def on_test_end(self, trainer, pl_module):
    logger.info("***** Test results *****")

    if pl_module.is_logger():
      metrics = trainer.callback_metrics

      # Log and save results to file
      output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
      with open(output_test_results_file, "w") as writer:
        for key in sorted(metrics):
          if key not in ["log", "progress_bar"]:
            logger.info("{} = {}\n".format(key, str(metrics[key])))
            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [62]:
class T5FineTuner(pl.LightningModule):
  def __init__(self, hparams):
    super(T5FineTuner, self).__init__()
    
    if type(hparams) is dict: 
      hparams = argparse.Namespace(**hparams)
    
    self.hparams = hparams
    self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
    self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)
  
  def is_logger(self):
    return self.trainer.global_rank <= 0
  
  def forward(
      self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
  ):
    return self.model(
        input_ids,
        attention_mask=attention_mask,
        decoder_input_ids=decoder_input_ids,
        decoder_attention_mask=decoder_attention_mask,
        labels=labels,
    )

  def _step(self, batch):
    labels = batch["target_ids"]
    labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

    outputs = self(
        input_ids=batch["source_ids"],
        attention_mask=batch["source_mask"],
        labels=labels,
        decoder_attention_mask=batch['target_mask']
    )

    loss = outputs[0]
    
    return loss

  def training_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('training_loss', loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}
  
  def training_epoch_end(self, outputs):
    avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
    self.log('avg_training_loss', avg_train_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"avg_train_loss": avg_train_loss}
    return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

  def validation_step(self, batch, batch_idx):
    loss = self._step(batch)
    self.log('val_loss', loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)
    tensorboard_logs = {"val_loss": loss}
    return {"val_loss": loss, "log": tensorboard_logs}
  
  def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    print(avg_loss)
    tensorboard_logs = {"val_loss": avg_loss}
    self.log('avg_val_loss', avg_loss, on_step=False, on_epoch=True, prog_bar=True, logger=True)

  def configure_optimizers(self):
    "Prepare optimizer and schedule (linear warmup and decay)"

    model = self.model
    no_decay = ["bias", "LayerNorm.weight"]
    optimizer_grouped_parameters = [
        {
            "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
            "weight_decay": self.hparams.weight_decay,
        },
        {
            "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
            "weight_decay": 0.0,
        },
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
    self.opt = optimizer
    return [optimizer]
  
  def optimizer_step(self,
                     epoch=None, 
                     batch_idx=None, 
                     optimizer=None, 
                     optimizer_idx=None, 
                     optimizer_closure=None, 
                     on_tpu=None, 
                     using_native_amp=None, 
                     using_lbfgs=None
                     ):

    optimizer.step(closure=optimizer_closure)
    optimizer.zero_grad()
    self.lr_scheduler.step()

  def train_dataloader(self):
    train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
    dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True, num_workers=4)
    t_total = (
        (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
        // self.hparams.gradient_accumulation_steps
        * float(self.hparams.num_train_epochs)
    )
    scheduler = get_linear_schedule_with_warmup(
        self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
    )
    self.lr_scheduler = scheduler
    return dataloader

  def val_dataloader(self):
    val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="val", args=self.hparams)
    return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [63]:
# The below code is adapted from:
# https://github.com/huggingface/transformers/blob/master/examples/multiple-choice/utils_multiple_choice.py

@dataclass(frozen=True)
class InputExample:
    """
    A single training/test example for multiple choice
    Args:
        example_id: Unique id for the example.
        question: string. The untokenized text of the second sequence (question).
        contexts: list of str. The untokenized text of the first sequence (context of corresponding question).
        endings: list of str. multiple choice's options. Its length must be equal to contexts' length.
        label: (Optional) string. The label of the example. This should be
        specified for train and dev examples, but not for test examples.
    """

    name: str
    ingredients: str
    steps: List[str]
    label: Optional[str]

class Split(Enum):
    train = "train"
    dev = "dev"
    test = "test"

class DataProcessor:
    """Base class for data converters for multiple choice data sets."""

    def get_train_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the train set."""
        raise NotImplementedError()

    def get_dev_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the dev set."""
        raise NotImplementedError()

    def get_test_examples(self, data_dir):
        """Gets a collection of `InputExample`s for the test set."""
        raise NotImplementedError()

    def get_labels(self):
        """Gets the list of labels for this data set."""
        raise NotImplementedError()

class RecipeProcessor(DataProcessor):
    """Processor for the SWAG data set."""

    def get_train_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} train".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "train.csv")), "train")

    def get_dev_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        return self._create_examples(self._read_csv(os.path.join(data_dir, "val.csv")), "dev")

    def get_test_examples(self, data_dir):
        """See base class."""
        logger.info("LOOKING AT {} dev".format(data_dir))
        raise ValueError(
            "For swag testing, the input file does not contain a label column. It can not be tested in current code"
            "setting!"
        )
        return self._create_examples(self._read_csv(os.path.join(data_dir, "test.csv")), "test")

    def get_labels(self):
        """See base class."""
        return ["0", "1", "2", "3"]

    def _read_csv(self, input_file):
        with open(input_file, "r", encoding="utf-8") as f:
            return list(csv.reader(f))

    def _create_examples(self, lines: List[List[str]], type: str):
        """Creates examples for the training and dev sets."""

        examples = [
            InputExample(
                name=line[1],
                # common beginning of each
                # choice is stored in "sent2".
                ingredients=eval(line[3]),
                steps=eval(line[2]),
                label=eval(line[2]),
            )
            for line in lines[1:]  # we skip the line with the column names
        ]

        return examples

In [64]:
class RecipeDataset(Dataset):
    
    def __init__(self, tokenizer, data_dir, type_path,  max_len=512, mask_percent=0.4, mode='fill'):
        self.data_dir = data_dir
        self.type_path = type_path
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []
        self.ingredients = []
        self.names = []
        self.mode = mode
        self.mask_percent = mask_percent
        
        self.proc = RecipeProcessor()
        self._build()
  
    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask    = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, 
                "target_ids": target_ids, "target_mask": target_mask}
  
    def __len__(self):
        return len(self.inputs)
  
    def _build(self):
        if self.type_path == 'train':
            examples = self.proc.get_train_examples(self.data_dir)
        else:
            examples = self.proc.get_dev_examples(self.data_dir)
    
        for i, example in enumerate(examples):
            if i % 10000 == 0:
                print(i)
            self._create_features(example)
    
    def _create_shuffle_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          
          mask_words = np.round(0.5*total_words).astype(int)
          if np.random.uniform() < 0.5:
            input_words = words[:mask_words] + ['<extra_id_0>']
            label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']
          else:
            input_words = ['<extra_id_0>'] + words[mask_words:]
            label = ['<extra_id_0>'] + words[:mask_words] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_completion(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          input_words = []
          label = []
        else:
          mask_words = np.round(0.5*total_words).astype(int)
          input_words = words[:mask_words] + ['<extra_id_0>']
          label = ['<extra_id_0>'] + words[mask_words:] + ['<extra_id_1>']

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
  
    def _create_masks(self, steps):
        
        words = [word for step in steps for word in step.split(" ")]
        total_words = len(words)
        
        if total_words == 0:
          mask_words = 0
        else:
          mask_words = np.round(self.mask_percent*total_words).astype(int)

        mask_indices = np.random.choice(np.arange(total_words), mask_words)
        
        input_words = []
        label = []
        j = 0
        
        for i, word in enumerate(words):
            if i in mask_indices:
                # Add a sentinel token in place of the word to be masked
                input_words.append('<extra_id_{}>'.format(j))
                # Add this word's token to the label list
                label.append('<extra_id_{}>'.format(j))
                # Add the corresponding label to the list
                label.append(word)
                # Step the counter by 1
                j+=1
            # If this index is not among the sampled indices 
            # just append the word
            else:
                input_words.append(word)
         
        # Add in an extra ID token like the format says
        label.append('<extra_id_{}>'.format(j))

        label = " ".join(label)
        input_words = " ".join(input_words)
        
        return(input_words, label)
    
    def _create_features(self, example):
    
        ingredients_ = ",".join(example.ingredients)
        name_ = example.name
        
        if self.mode == 'fill':
          masked_, label_ = self._create_masks(example.steps)
        
        elif self.mode == 'complete':
          masked_, label_ = self._create_completion(example.steps)

        elif self.mode == 'shuffle_complete':
          masked_, label_ = self._create_completion(example.steps)  
                                        
        input_ = "name: %s  ingredients: %s masked: %s </s>" % (name_, ingredients_, masked_)
        target =  label_ + " </s>"
        
        # tokenize inputs
        tokenized_inputs = self.tokenizer.batch_encode_plus(
            [input_], max_length = 128, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )
        # tokenize targets
        tokenized_targets = self.tokenizer.batch_encode_plus(
            [target], max_length = 75, pad_to_max_length=True, truncation=True, return_tensors="pt"
        )

        self.inputs.append(tokenized_inputs)
        self.targets.append(tokenized_targets)
        self.ingredients.append(example.ingredients)
        self.names.append(example.name)

In [65]:
def generate_loop(checkpoint_dir='drive/MyDrive/recipe-generation/t5_fill_blanks/', 
                  checkpoint_name = 'exp_7_fb_ef.ckpt', 
                  experiment_name = 'experiment_7',
                  mode = 'fill'):
  '''
  ---------
  Generation loop
  --------
  '''
  checkpoint_path = os.path.join(checkpoint_dir, checkpoint_name)
  model = T5FineTuner.load_from_checkpoint(checkpoint_path)
  tokenizer = T5Tokenizer.from_pretrained("t5-base")

  dataset =  RecipeDataset(tokenizer, data_dir='drive/MyDrive/recipe-generation/data', type_path='val', mode=mode)
  loader = DataLoader(dataset, batch_size=32, num_workers=2)

  model.model.eval()
  outputs = []
  targets = []

  for i, batch in enumerate(tqdm(loader)):

    # Need to change max. length argument
    outs = model.model.generate(input_ids=batch['source_ids'], 
                                attention_mask=batch['source_mask'], 
                                max_length=75)
    dec = [tokenizer.decode(ids, skip_special_tokens=True) for ids in outs]
    target = [tokenizer.decode(ids, skip_special_tokens=True) for ids in batch["target_ids"]]

    if i%100 == 0:
      print(dec)
      print(target)

    outputs.extend(dec)
    targets.extend(target)

  results = [dataset.names, dataset.ingredients, outputs, targets]
  results_path = os.path.join(checkpoint_dir, '{}_outputs.pkl'.format(experiment_name))
  
  return(results) 

In [66]:
def write_results(results, results_path):
  '''
  Pickle and write results 
  to file
  '''
  with open(results_path, 'wb') as f:
    pickle.dump(results, f)
    f.close()

In [67]:
def read_results(results_path):
    '''
    ----------
    Load results from pkl. file
    -----------
    '''
    # Open statement within context
    with open(results_path, 'rb') as f:
      # Load statement
      results = pickle.load(f)
    # Return statement
    return(results)

In [68]:
checkpoint_dir='drive/MyDrive/recipe-generation/t5_fill_blanks/'

In [69]:
checkpoint_name = 'exp_5_rc_both.ckpt' 
experiment_name = 'experiment_5'
mode = 'shuffle_complete'
results_path = os.path.join(checkpoint_dir, '{}_outputs.pkl'.format(experiment_name))

results = generate_loop(checkpoint_name=checkpoint_name, experiment_name=experiment_name, mode=mode)

0


/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:2079: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."


10000



  0%|          | 1/568 [00:06<1:00:26,  6.40s/it]

[', about 5 minutes remove from the heat and stir in the pomegranate seeds let cool to room temperature, then refrigerate until ready to use, up to 2 days', 'and cover with a damp cloth let rise until doubled in size, about 30 minutes preheat oven to 375 degrees f fry until golden brown, about 5 minutes drain on paper towels serve warm with butter, jam or honey, or ketchup', '/ 2 inch thick slices and add to soup simmer gently for 1 hour or until cabbage is tender serve with crusty bread', 'into a greased 9-inch pie plate bake at 350 degrees for 40 minutes or until knife inserted near center comes out clean cool on wire rack', ', and bring to a boil reduce heat to low and simmer for 30 minutes add spinach and simmer for 5 minutes more season with salt and pepper to taste', 'in the pan pour sweetened condensed milk over all bake for 20 minutes, or until top is golden brown cool completely and cut into bars store in airtight container in refrigerator', 'in a small bowl, combine vanilla a

...


 71%|███████   | 401/568 [28:35<11:47,  4.24s/it]

['the flour and chocolate chips in a small bowl stir the dry ingredients into the wet ingredients until just combined drop by rounded teaspoonfuls onto a greased cookie sheet bake for 10-12 minutes or until golden brown cool on a wire rack', 'and water in a saucepan bring to a boil over medium heat boil for 2 minutes, stirring constantly remove from the heat stir in cashews spread the mixture over the baked crust bake for 20 to 25 minutes or until the filling is bubbly and the edges are golden brown cool completely in the pan on a wire rack', 'to 3 / 4 hours, remove from heat and let cool to room temperature add papaya and papaya seeds and mix well', 'a 1 / 2 teaspoon of royal or acty royal or a single-acting baking powder, use 1 teaspoon of cornstarch', "m&m's, chocolate chips and nuts drop by rounded teaspoonfuls onto ungreased cookie sheet bake for 10-12 minutes or until edges are lightly browned cool on cookie sheet for 2 minutes before removing to wire racks", ', cover and let ris


 88%|████████▊ | 501/568 [35:41<04:46,  4.28s/it]

['minutes or until heated through add haloumi to pan cook, stirring, for 2 minutes or until haloumi is golden and cheese is melted stir in lemon rind and juice season to taste with salt and pepper spoon haloumi onto lettuce leaves and serve', ', salt, pepper, and water bring to a boil, reduce heat, cover and simmer for 1 hour, stirring occasionally', 'add macaroni, beans, carrots and cheese stir to combine cover and chill for at least 1 hour before serving', 'be dissolved add remaining ingredients and blend until smooth pour into glasses and serve', 'with the marinade refrigerate for at least 2 hours, turning occasionally grill or broil the steaks to desired doneness, basting with the marinade every 5 minutes, until the steaks are cooked to your liking, about 5 minutes per side', 'cornstarch, baking soda, salt, evaporated milk and vanilla mix well pour into prepared baking dish and bake for 25-30 minutes cool and cut', 'oil mix well store in refrigerator for up to 2 weeks', ', granulat


100%|██████████| 568/568 [40:24<00:00,  4.27s/it]


In [72]:
checkpoint_name = 'exp_5_rc_both.ckpt' 
experiment_name = 'experiment_5'
mode = 'shuffle_complete'
results_path = os.path.join(checkpoint_dir, '{}_outputs.pkl'.format(experiment_name))

In [74]:
write_results(results, results_path)